In [44]:
#!conda install -y geopandas
#!conda install -y -c conda-forge folium
#!pip install OSMPythonTools
#!pip install openrouteservice
#!pip install haversine

In [64]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import folium
from haversine import haversine
from os import listdir


%matplotlib inline

In [84]:
listdir()

['made-2020-geo-hw',
 'Untitled.ipynb',
 'data.zip',
 '.ipynb_checkpoints',
 'drivers_with_gps_and_sl_problems',
 'drivers_with_gps_and_sl_problems.zip',
 'data',
 'Deep learning for fraud detection in retail transactions | by Ahmad Khodayari | Walmart Global Tech Blog | Medium.pdf',
 'img-2021-10-19-23-08-23.png']

In [86]:
pd.DataFrame(listdir("./drivers_with_gps_and_sl_problems"))

,0
0,2025312357143911724
1,1293535473153840726
2,-8729475955540025841
3,-7745607963253656189
4,-2132251359044308970
5,8569741546477592565
6,-8915411586673655243
7,-7078426713190014901
8,-4687681644253937266
9,-2988946964244641879


In [184]:
path = "drivers_with_gps_and_sl_problems/2765355583032989810"

In [185]:
xyz = pd.read_csv(f"./{path}/accelerometer.csv", parse_dates=[0])
xyz

,time,x,y,z,lat,lon
0,2021-02-10 13:02:04.325071+03:00,-0.084916,-0.011960,9.904077,55.750522,37.824699
1,2021-02-10 13:02:04.325576+03:00,-0.087308,-0.019136,9.912448,55.750547,37.824350
2,2021-02-10 13:02:04.330045+03:00,-0.089700,-0.009568,9.914840,55.751304,37.819936
3,2021-02-10 13:02:04.330464+03:00,-0.092092,-0.002392,9.917232,55.751322,37.819589
4,2021-02-10 12:59:04.414380+03:00,-0.087308,0.004784,9.912448,55.750093,37.833083
...,...,...,...,...,...,...
9592,2021-02-12 18:50:28.136986+03:00,-0.040664,0.088504,9.890921,55.743754,37.677328
9593,2021-02-12 18:50:28.137678+03:00,-0.027508,0.081328,9.900488,55.743648,37.677244
9594,2021-02-12 18:50:28.141834+03:00,-0.044252,0.078936,9.898096,55.742876,37.676549
9595,2021-02-12 18:50:28.133933+03:00,-0.047840,0.082524,9.898096,55.744045,37.677605


In [186]:
gps = pd.read_csv(f"./{path}/track.csv", parse_dates=[0, 3])
gps

,time,lat,lon,gps_time
0,2021-02-10 11:48:58,55.687807,37.900947,2021-02-10 11:48:58
1,2021-02-10 11:49:04,55.687807,37.900946,2021-02-10 11:49:04
2,2021-02-10 11:49:06,55.687807,37.900946,2021-02-10 11:49:06
3,2021-02-10 11:49:07,55.687807,37.900946,2021-02-10 11:49:07
4,2021-02-10 11:49:11,55.687808,37.900946,2021-02-10 11:49:11
...,...,...,...,...
19960,2021-02-11 18:46:26,55.739710,37.676324,2021-02-11 18:46:26
19961,2021-02-11 18:46:28,55.739710,37.676324,2021-02-11 18:46:28
19962,2021-02-11 18:46:29,55.739710,37.676324,2021-02-11 18:46:29
19963,2021-02-11 18:46:30,55.739710,37.676324,2021-02-11 18:46:29


In [187]:
xyz["time"].sort_values().diff().describe()

count                         9596
mean     0 days 00:00:20.229627580
std      0 days 00:15:46.930968418
min         0 days 00:00:00.000345
25%         0 days 00:00:00.000425
50%         0 days 00:00:00.000542
75%         0 days 00:00:00.000626
max         0 days 20:00:57.673498
Name: time, dtype: object

In [188]:
gps["time"].sort_values().diff().describe()

count                        19964
mean     0 days 00:00:05.582648767
std      0 days 00:06:41.116887119
min                0 days 00:00:00
25%                0 days 00:00:02
50%                0 days 00:00:03
75%                0 days 00:00:03
max                0 days 15:44:30
Name: time, dtype: object

In [189]:
center = (xyz["lat"].mean(), xyz["lon"].mean())
center

(55.75525605137022, 37.76972022538293)

In [190]:
m = folium.Map(location=center,
              zoom_start=10, legend=True)

gps_track = gps.sort_values(by="gps_time").loc[:,["lat", "lon"]]
folium.PolyLine(gps_track,
                color='blue',
                weight=10,
                label="tracker",
                opacity=0.8).add_to(m)

xyz_track = xyz.sort_values(by="time").loc[:,["lat", "lon"]]

folium.PolyLine(xyz_track,
                color='red',
                weight=5,
                label="accel",
                opacity=0.8).add_to(m)



m

In [152]:
xyz['time'].dt.floor(freq='H').unique()

<DatetimeArray>
['2021-05-24 13:00:00+03:00', '2021-05-24 14:00:00+03:00',
 '2021-05-24 15:00:00+03:00', '2021-05-24 16:00:00+03:00',
 '2021-05-24 17:00:00+03:00', '2021-05-24 18:00:00+03:00',
 '2021-05-24 19:00:00+03:00', '2021-05-24 20:00:00+03:00',
 '2021-05-24 21:00:00+03:00', '2021-05-24 23:00:00+03:00',
 '2021-05-25 11:00:00+03:00', '2021-05-25 12:00:00+03:00']
Length: 12, dtype: datetime64[ns, pytz.FixedOffset(180)]

In [153]:
xyz['time'].dt.floor(freq='min').unique()

<DatetimeArray>
['2021-05-24 13:40:00+03:00', '2021-05-24 13:39:00+03:00',
 '2021-05-24 13:41:00+03:00', '2021-05-24 13:42:00+03:00',
 '2021-05-24 13:48:00+03:00', '2021-05-24 13:45:00+03:00',
 '2021-05-24 13:47:00+03:00', '2021-05-24 13:46:00+03:00',
 '2021-05-24 13:49:00+03:00', '2021-05-24 13:50:00+03:00',
 ...
 '2021-05-25 12:46:00+03:00', '2021-05-25 12:50:00+03:00',
 '2021-05-25 12:51:00+03:00', '2021-05-25 12:20:00+03:00',
 '2021-05-25 12:38:00+03:00', '2021-05-25 12:41:00+03:00',
 '2021-05-25 12:47:00+03:00', '2021-05-24 18:01:00+03:00',
 '2021-05-24 18:20:00+03:00', '2021-05-24 16:00:00+03:00']
Length: 373, dtype: datetime64[ns, pytz.FixedOffset(180)]

In [154]:
xyz_coord = xyz.sort_values(by="time")\
    .loc[:,["lat", "lon"]]

xyz_coord
#     .apply(lambda x: haversine(x[0], x[1], x[0].shift(), x[1].shift()))

,lat,lon
6703,55.712708,37.607547
2,55.712708,37.607547
5107,55.712708,37.607547
1795,55.712708,37.607547
1794,55.712708,37.607547
...,...,...
8222,55.679031,37.585934
4888,55.678979,37.585847
8221,55.678906,37.585648
6659,55.678862,37.585531


In [155]:
xyz_coord.shift(-1)

,lat,lon
6703,55.712708,37.607547
2,55.712708,37.607547
5107,55.712708,37.607547
1795,55.712708,37.607547
1794,55.712708,37.607547
...,...,...
8222,55.678979,37.585847
4888,55.678906,37.585648
8221,55.678862,37.585531
6659,55.678810,37.585435
